In [2]:
!pip install transformers

     |████████████████████████████████| 1.5 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 883 kB 3.5 MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893259 sha256=eca2583b1d3326e50f88ad8136e6e6f31a080520cfb84498ca65e4dca62d0dcc
  Stored in directory: /Users/ardsnijders/Library/Caches/pip/wheels/7b/78/f4/27d43a65043e1b75dbddaa421b573eddc67e712be4b1c80677
Successfully built sacremoses
You should consider upgrading via the '/Users/ardsnijders/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [4]:
import torch
from transformers import AutoModel, AutoTokenizer, BertTokenizer

torch.set_grad_enabled(False)

In [5]:
# Store the model we want to use
MODEL_NAME = "bert-base-cased"

# We need to create the model and tokenizer
model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [35]:
# print(model.parameters)
from pprint import pprint
# pprint(vars(model._modules()))
print(model._modules['encoder'].layer[0])

BertLayer(
  (attention): BertAttention(
    (self): BertSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
  )
  (output): BertOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)


In [37]:
print(model._modules['encoder'].layer[0].attention.self)

BertSelfAttention(
  (query): Linear(in_features=768, out_features=768, bias=True)
  (key): Linear(in_features=768, out_features=768, bias=True)
  (value): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


In [38]:
#put the model in eval mode
model.eval()

sent = "The animal didn't cross the street because it was too tired"
#Add CLS and SEP Tokens considering it as one sentence
sent = "[CLS] " + sent + " [SEP]"

#apply tokenizer and get ids
tokenized_text = tokenizer.tokenize(sent)
tokenized_text_ids = tokenizer.convert_tokens_to_ids(tokenized_text)

#Convert to torch tensor
input = torch.tensor(tokenized_text_ids)

print("input.shape: ", input.shape) #torch.Size([15]) , N=15 here
input = input.unsqueeze(dim=0) #add a dimension 

#Fwd pass - get output
output = model(input) 

input.shape:  torch.Size([15])


In [ ]:
def get_attentions(outputs, layer=0, attention_head=0, avg=False):
    '''
    get the particular output for a particular layer and attention head
    layer -> 0 to 11
    attention_head -> 0 to 11
    '''
    if avg:
    #avg over all attention heads in a layer
    return outputs[layer].squeeze(0).mean(dim=0)

    #return values for a particular attention head inside a specific layer
    return outputs[layer].squeeze(0)[attention_head]

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.5687,  0.1763,  0.0672,  ..., -0.3716,  0.1667, -0.1321],
         [-0.0058, -0.0217,  0.1018,  ...,  0.1843, -0.3020,  0.3822],
         [ 0.3899,  0.0632, -0.0706,  ..., -0.3645, -0.1812, -0.2884],
         ...,
         [ 0.4359, -0.0335,  0.0881,  ..., -0.5215,  0.6530,  0.3643],
         [ 0.1118, -0.3311,  0.0931,  ..., -0.8827, -0.0320, -0.1574],
         [ 1.5229,  0.1413, -0.2243,  ..., -0.5624,  0.0394, -0.7298]]]), pooler_output=tensor([[-6.6049e-01,  4.0153e-01,  9.9965e-01, -9.8873e-01,  9.5425e-01,
          9.2909e-01,  9.8409e-01, -9.9621e-01, -9.6883e-01, -6.4378e-01,
          9.7436e-01,  9.9693e-01, -9.9912e-01, -9.9950e-01,  9.0103e-01,
         -9.6699e-01,  9.8516e-01, -5.1332e-01, -9.9988e-01, -8.8926e-01,
         -6.0914e-01, -9.9966e-01,  3.5943e-01,  9.7838e-01,  9.7174e-01,
          5.3969e-02,  9.7922e-01,  9.9986e-01,  9.2669e-01, -6.1083e-01,
          1.7518e-01, -9.8255e-01,  